In [1]:
%pdb

Automatic pdb calling has been turned ON


In [2]:
from __future__ import division
import numpy as np
import scipy.spatial as spatial
import sys
import time
import os
import pandas as pd
import time
from joblib import Memory, Parallel, delayed
# from https://pypi.python.org/pypi/bintrees/2.0.2
from bintrees import FastAVLTree, FastBinaryTree
# from https://github.com/juhuntenburg/brainsurfacescripts
from vtk_rw import read_vtk, write_vtk
from graphs import graph_from_mesh

# function returning generator for sub,hemi tuples fro joblib
def tupler(subjects, hemis):
    for s in subjects:
        for h in hemis:
            yield (s, h)

# function to find unlabelled neighbours of a node in the graph
# and add them to the tree correctly
def add_neighbours(node, length, graph, labels, tree):
    # find direct neighbours of the node
    neighbours = np.array(graph.neighbors(node))
    # check that they don't already have a label
    unlabelled = neighbours[np.where(labels[neighbours][:,1]==-1)[0]]
    # insert source neighbour pair with edge length to tree
    for u in unlabelled:
        new_length = length + graph[node][u]['length']
        tree.insert(new_length,(node, u))
    
    return tree

# function to write time and message to log file
def log(log_file, message, logtime=True):
    with open(log_file, 'a') as f:
        if logtime:
            f.write(time.ctime()+'\n')
        f.write(message+'\n')

In [3]:
sub='PL6T'
hemi='rh'

In [4]:
# log different steps in logfile
log_file = '/scr/ilz3/myelinconnect/working_dir/complex_to_simple/testing_AVL_log_worker_%s.txt'%(str(os.getpid()))
log(log_file, 'Processing %s %s'%(sub, hemi))

In [5]:
# not optimal to have this hardcoded inside the function
# figure out how to pipe with parallel and move outside
complex_file = '/scr/ilz3/myelinconnect/struct/surf_%s/orig/mid_surface/%s_%s_mid.vtk'
simple_file = '/scr/ilz3/myelinconnect/groupavg/indv_space/%s/lowres_%s_d_def.vtk'# version d
label_file = '/scr/ilz3/myelinconnect/all_data_on_simple_surf/labels/AVL/testing_%s_%s_highres2lowres_labels.npy'
surf_label_file = '/scr/ilz3/myelinconnect/all_data_on_simple_surf/labels/AVL/testing_%s_%s_highres2lowres_labels.vtk'

In [6]:
# load the meshes
log(log_file, '...loading data', logtime=False)
complex_v, complex_f, complex_d = read_vtk(complex_file%(hemi, sub, hemi))
simple_v, simple_f, simple_d = read_vtk(simple_file%(sub, hemi))

In [23]:
def find_voronoi_seeds(simple_vertices, complex_vertices):

    # make array for writing in final voronoi seed indices
    voronoi_seed_idx = np.zeros((simple_vertices.shape[0],), dtype='int64')-1
    missing = np.where(voronoi_seed_idx==-1)[0].shape[0]
    mapping_single = np.zeros_like(voronoi_seed_idx)

    neighbours = 0
    col = 0

    while missing != 0:

        print 'increasing neighbours'
        neighbours += 100

        print 'kdtree'
        # find those points on the individuals complex mesh that correspond best
        # to the simplified group mesh in subject space
        inaccuracy, mapping  = spatial.KDTree(complex_vertices).query(simple_vertices, k=neighbours)
        nearest_idx = np.copy(mapping[:,0])

        
        print 'loop' 
        while col < neighbours:

            print col

            # find all missing voronoi seed indices
            missing_idx = np.where(voronoi_seed_idx==-1)[0]
            missing = missing_idx.shape[0]
            print 'missing '+str(missing)

            if missing == 0:
                break
            else:
                # for missing entries fill in next neighbour
                mapping_single[missing_idx]=np.copy(mapping[missing_idx,col])

                # find unique values in mapping_single
                unique, double_idx = np.unique(mapping_single, return_inverse = True)
                print 'unique '+str(unique.shape[0])

                # empty voronoi seed index
                voronoi_seed_idx = np.zeros((simple_vertices.shape[0],), dtype='int64')-1
                # fill voronoi seed idx with unique values
                for u in range(unique.shape[0]):
                    # find the indices of this value in mapping
                    entries = np.where(double_idx==u)[0]
                    # set the first entry to the value
                    voronoi_seed_idx[entries[0]] = unique[u]

                # go to next column
                col += 1 
                
    return voronoi_seed_idx, nearest_idx

In [24]:
voronois
= find_voronoi_seeds(simple_v, complex_v)

increasing neighbours
kdtree
loop
0
missing 75589
64492
1
missing 11097
69112
2
missing 6477
70617
3
missing 4972
71423
4
missing 4166
71917
5
missing 3672
72273
6
missing 3316
72543
7
missing 3046
72771
8
missing 2818
72960
9
missing 2629
73108
10
missing 2481
73245
11
missing 2344
73365
12
missing 2224
73442
13
missing 2147
73513
14
missing 2076
73585
15
missing 2004
73645
16
missing 1944
73703
17
missing 1886
73768
18
missing 1821
73824
19
missing 1765
73867
20
missing 1722
73911
21
missing 1678
73957
22
missing 1632
74000
23
missing 1589
74044
24
missing 1545
74082
25
missing 1507
74115
26
missing 1474
74141
27
missing 1448
74170
28
missing 1419
74197
29
missing 1392
74222
30
missing 1367
74250
31
missing 1339
74279
32
missing 1310
74305
33
missing 1284
74333
34
missing 1256
74352
35
missing 1237
74379
36
missing 1210
74401
37
missing 1188
74419
38
missing 1170
74437
39
missing 1152
74458
40
missing 1131
74477
41
missing 1112
74499
42
missing 1090
74513
43
missing 1076
74534
44
mis

In [42]:
print voronois.shape[0]
print np.unique(voronois).shape[0]

75589
75589


In [25]:
voronoi_seed_idx = np.squeeze(voronois)
voronoi_seed_coord = complex_v[voronoi_seed_idx]
dist = np.linalg.norm((voronoi_seed_coord - simple_v), axis=1)

In [47]:
inaccuracy, mapping  = spatial.KDTree(complex_v).query(simple_v, k=3)

In [49]:
inaccuracy[:,0].shape

(75589, 3)

In [68]:
np.where(np.abs((np.abs(dist)-np.abs(inaccuracy[:,0]))) > 3)

(array([27871, 27872, 27874, 27884, 27885, 28395, 28396, 28398, 28399,
        28404, 28405, 28406, 28407, 28409, 28413, 28414, 28415, 28417,
        28930, 28933, 28934, 28940, 28948, 29472, 29473, 29475, 29481,
        29482, 29483, 29484, 30057, 30058, 30059, 30065, 30072, 30083,
        30084, 30086, 30699, 30701, 30703, 30704, 30710, 30712, 30716,
        30717, 31356, 31357, 31361, 32023]),)

In [69]:
dist[29472]

5.4688253942780118

In [70]:
inaccuracy[:,0][29472]

2.1932451879805934

In [79]:
if ((np.mean(dist)-np.mean(inaccuracy[:,0])>0.01)):
    print 'no'
else:
    print 'yes'

no


In [63]:
print dist.mean()
print dist.min()
print dist.max()

0.922277554678
0.0107682914151
7.32925310946


In [62]:
print inaccuracy[:,0].mean()
print inaccuracy[:,0].min()
print inaccuracy[:,0].max()

0.874485641749
0.0107682914151
7.30263559429


In [41]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('poster')
plt.hist(inaccuracy, label='nearest', alpha=0.5, color='b', bins=100)
plt.hist(dist, label='unique', alpha=0.5, color='r', bins=100)
plt.legend()
plt.show()